<a href="https://colab.research.google.com/github/ras0k/auto-lyrics/blob/main/Auto_Lyrics_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Whisper transcription (avec option de langue)
import subprocess
import sys
import os

# Install necessary packages quietly
print("Installing packages, please wait...")
!pip install yt-dlp > /dev/null 2>&1
!pip install git+https://github.com/openai/whisper.git > /dev/null 2>&1
!pip install ipywidgets > /dev/null 2>&1

# Import necessary modules
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import output
from google.colab import files

# Clear the output to display widgets first
clear_output(wait=True)

# --- WIDGETS CREATION ---

# 1. Source Option
option_dropdown = widgets.Dropdown(
    options=['Download from YouTube URLs', 'Upload audio files'],
    value='Download from YouTube URLs',
    description='Source:',
    layout={'width': 'max-content'},
    style={'description_width': 'initial'}
)

# 2. Language Option (New!)
# Whisper accepts full names (French) or codes (fr).
common_languages = [
    ('Auto detection', 'auto'),
    ('French (Français)', 'French'),
    ('English', 'English'),
    ('Spanish (Espagnol)', 'Spanish'),
    ('German (Allemand)', 'German'),
    ('Italian (Italien)', 'Italian'),
    ('Portuguese (Portugais)', 'Portuguese'),
    ('Dutch (Néerlandais)', 'Dutch'),
    ('Japanese (Japonais)', 'Japanese'),
    ('Chinese (Chinois)', 'Chinese'),
    ('Russian (Russe)', 'Russian')
]

language_dropdown = widgets.Dropdown(
    options=common_languages,
    value='auto',
    description='Language:',
    layout={'width': 'max-content'},
    style={'description_width': 'initial'}
)

# 3. Input Text Area
text_area = widgets.Textarea(
    value='',
    placeholder='Enter YouTube URLs separated by new lines',
    description='URLs:',
    layout={'width': '100%', 'height': '150px'},
    style={'description_width': 'initial'}
)

# 4. Process Button
process_button = widgets.Button(
    description="Start Transcription",
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    layout={'width': '200px'}
)

# Output area for logs
output_area = widgets.Output()

# --- LOGIC ---

def on_process_clicked(b):
    with output_area:
        clear_output(wait=True)

        def run_command(command):
            """Run a shell command and display output in real-time."""
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
            for line in iter(process.stdout.readline, ''):
                print(line, end='')
                sys.stdout.flush()
            process.wait()

        def transcribe_audio(file_paths, lang_code):
            for i, file_path in enumerate(file_paths, 1):
                if file_path.strip():
                    print(f"\n--- Transcribing: {file_path} ---")

                    # Build the Whisper command
                    # We use the 'large' model for best accuracy
                    base_cmd = f"whisper '{file_path}' --model large"

                    # Add language flag if not auto
                    if lang_code != 'auto':
                        cmd = f"{base_cmd} --language {lang_code}"
                        print(f"Language forced to: {lang_code}")
                    else:
                        cmd = base_cmd
                        print("Language detection: Auto")

                    run_command(cmd)
                    print(f"Transcription completed for {file_path}\n")

        def download_and_transcribe(urls, lang_code):
            urls = urls.strip().split('\n')
            file_paths = []
            print("--- Downloading ---")
            for i, url in enumerate(urls, 1):
                if url.strip():
                    # clean filename using strict ascii or specific format to avoid shell issues
                    audio_file = f"audio_download_{i}"
                    print(f"Processing URL: {url}")

                    # yt-dlp command: extract audio to mp3
                    yt_dlp_command = f"yt-dlp -x -o '{audio_file}.%(ext)s' --audio-format mp3 --force-overwrites '{url}'"
                    run_command(yt_dlp_command)

                    # Find the file we just downloaded
                    downloaded_files = [f for f in os.listdir('.') if f.startswith(audio_file)]
                    file_paths.extend(downloaded_files)

            # Start transcription
            transcribe_audio(file_paths, lang_code)

        def upload_and_transcribe(lang_code):
            print("Please upload your audio files using the widget below:")
            uploaded = files.upload()
            if not uploaded:
                print("No files uploaded.")
                return
            file_paths = list(uploaded.keys())
            transcribe_audio(file_paths, lang_code)

        # Main execution block inside the button click
        lang_selection = language_dropdown.value

        if option_dropdown.value == 'Download from YouTube URLs':
            if text_area.value.strip():
                download_and_transcribe(text_area.value, lang_selection)
            else:
                print("Error: Please enter at least one YouTube URL.")
        else:
            upload_and_transcribe(lang_selection)

process_button.on_click(on_process_clicked)

# Function to update displayed widgets based on selection
def on_option_change(change):
    if change['new'] == 'Download from YouTube URLs':
        text_area.layout.display = 'block'
    else:
        text_area.layout.display = 'none'

option_dropdown.observe(on_option_change, names='value')

# Initial display of widgets
# Using VBox for a cleaner vertical layout
ui = widgets.VBox([
    widgets.HBox([option_dropdown, language_dropdown]), # Put dropdowns side by side
    text_area,
    process_button,
    output_area
])

display(ui)

In [ ]:
# @title Collect all .txt files and zip them (optional)

import os
import zipfile

def zip_txt_files(directory, zip_filename):
  """Zips all .txt files in a directory."""
  with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(directory):
      for file in files:
        if file.endswith(".txt"):
          filepath = os.path.join(root, file)
          zipf.write(filepath, os.path.relpath(filepath, directory))

# Example usage:
zip_txt_files(".", "txt_files.zip")  # Zip all .txt files in the current directory
print("TXT files zipped successfully.")

In [ ]:
# @title Delete all files to reset

import os
import glob

# Remove files starting with "audio"
files_to_remove = glob.glob('audio*')
for file in files_to_remove:
    try:
        os.remove(file)
        print(f"Removed: {file}")
    except OSError as e:
        print(f"Error removing {file}: {e}")

print("File cleanup complete.")